In [ ]:
import cv2
import tkinter as tk
from tkinter import messagebox
import threading
import sys
from deepface import DeepFace

# Variable para almacenar la ubicación de la foto
ubicacion = ""

# Crear el objeto de captura de video
capture = cv2.VideoCapture(0)

# Verificar si la cámara se abrió correctamente
if not capture.isOpened():
    raise IOError("No se puede abrir la cámara")

# Función para tomar la foto y guardarla
def tomar_foto():
    global ubicacion
    ret, frame = capture.read()
    ubicacion = "/home/diego/Pictures/foto.jpg"
    cv2.imwrite(ubicacion, frame)
    messagebox.showinfo("Foto tomada", "La foto se guardó correctamente.")
    buscar_foto()

def buscar_foto():
    global ubicacion
    if ubicacion:
        df = DeepFace.find(img_path=ubicacion, db_path='/home/diego/Pictures/dataset/',enforce_detection=False)
        print(df)
        messagebox.showinfo("Resultado de la búsqueda", str(df))
    else:
        messagebox.showinfo("Resultado de la búsqueda", "No se pudo realizar la búsqueda porque la ubicación de la imagen está vacía.")

# Función para finalizar la aplicación
def finalizar():
    capture.release()
    cv2.destroyAllWindows()
    sys.exit()

# Función para ejecutar el bucle de eventos de Tkinter en un hilo separado
def ejecutar_interfaz():
    ventana = tk.Tk()

    boton_tomar_foto = tk.Button(ventana, text="Tomar Foto", command=tomar_foto)
    boton_tomar_foto.pack()

    boton_finalizar = tk.Button(ventana, text="Finalizar", command=finalizar)
    boton_finalizar.pack()

    ventana.mainloop()

# Función para ejecutar el stream de video en un hilo separado
def ejecutar_video():
    while True:
        ret, frame = capture.read()
        cv2.imshow("Camera", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Iniciar el hilo para ejecutar la interfaz de usuario
hilo_interfaz = threading.Thread(target=ejecutar_interfaz)
hilo_interfaz.start()

# Iniciar el hilo para ejecutar el stream de video
hilo_video = threading.Thread(target=ejecutar_video)
hilo_video.start()

# Esperar a que los hilos terminen
hilo_interfaz.join()
hilo_video.join()